In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

# db management libraries
import pymysql
from controller import MysqlController

In [2]:
df = pd.DataFrame(columns = ['name', 'category', 'address', 'program'])
for file in os.listdir('/Users/yejin/Downloads/NAVERWORKS/'):
    if file.endswith('.csv'):
        dd = pd.read_csv('/Users/yejin/Downloads/NAVERWORKS/'+file, sep = ';')
        dd['program'] = file[:file.index('_')]
        # df.iloc[len(df), :] = dd
        df = pd.concat([df, dd])

In [3]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
# # 프로그램명 합치기
# df.program += ', '
# df = df.groupby(by= ['name', 'category', 'address']).sum()
# df.replace(', $', "", regex=True)

# one-hot encoding
DF = pd.concat([df, pd.get_dummies(df.program)], axis = 1)

In [5]:
DF = DF.groupby(by = ['name', 'category', 'address']).sum()
DF.reset_index(inplace=True)

In [6]:
# 연결
with open('../connection.txt', "r") as f:
                connect_info = list(map(lambda x: x.strip(), f.read().split(",")))
server = MysqlController(*connect_info)

In [ ]:
server.conn.close()

In [7]:
DF

,name,category,address,2TV생생정보,6시내고향,맛있는녀석들,모닝와이드,백종원의골목식당,생방송오늘저녁,생방송투데이,생활의달인,수요미식회,전지적참견시점
0,089coffee&bakery,베이커리,서울 강서구 화곡동 1116-3 1층 089coffee&bakery,0,0,0,0,0,0,1,0,0,0
1,100년의 꿈 제주 해물탕 벧엘본점,"매운탕,해물탕",서울 강동구 둔촌동 603-1 우리빌딩,1,0,0,0,0,1,0,0,0,0
2,100호,소고기구이,서울 금천구 독산동 292-10,0,0,0,0,0,0,1,0,0,0
3,101번지남산돈까스 남산 본점,101번지남산돈까스,서울 중구 남산동2가 49-24,1,0,0,0,0,0,0,0,0,0
4,102베이커리,베이커리,서울 강서구 화곡동 1073-10 1층 102호 화곡역 케이에스프리미어 1층 102호,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,히말라야어죽,"해물,생선요리",서울 마포구 도화동 329 도화주택,1,0,0,0,0,0,0,0,0,0
4044,히메시야,일식당,서울 마포구 상수동 313-1 지층 히메시야,0,0,0,0,0,0,0,1,0,0
4045,히츠지야 연남점,"육류,고기요리",서울 마포구 연남동 229-55,0,0,0,0,0,1,0,0,0,0
4046,히피스베이글,베이커리,서울 강북구 우이동 72-78,0,0,0,0,0,0,0,1,0,0


In [8]:
for line in DF.to_dict(orient='records'):
    placeholders = ','.join(['%s']*len(line))
    # server.insert? column name error '생활의달인' 없음
    sql_command = """INSERT INTO TV_restaurants(name, category, address, 2TV생생정보, 6시내고향, 백종원의골목식당, 
 맛있는녀석들, 모닝와이드, 생방송오늘저녁, 생방송투데이, 생활의달인, 수요미식회, 전지적참견시점, updated_at) VALUES(%s, now())"""%(placeholders)
    server.curs.execute(sql_command, tuple(str(val) for val in line.values()))
    server.conn.commit()